In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu
Created on Jan. 14, 2024
Last modified on Jan. 14, 2024

Convert the stellar yield file from Sukhbold et al. (2016) to the format more convenient for 
further IMF determination.
"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import quad
import pyatomdb
import re
import os
from functools import reduce
%matplotlib widget

In [3]:
def get_S16_single_yields(file):
    """
    Get the yileds from S16.
    
    Parameters
    ----------
    file : str
        The file name of the yield table.
    
    Returns
    -------
    yields : dataframe
        The dataframe of the yields.
        Its indice are "H, He, Li ...".
        Its column is "yield (ejecta+wind)", whose name is the initial mass.
    """
    # Get the number between 's' and '.yield_table', which is the initial mass.
    pattern = re.compile(r's(\d+(?:\.\d+)?).yield_table')
    match = pattern.search(file)
    m_ini = match.group(1)
    if 'implosions_W18' in file:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('wind', 'f8')])

        y = pd.DataFrame(y['wind'], index=y['isotope'], columns=[m_ini])
    else:
        y = np.loadtxt(file, skiprows=1, dtype=[('isotope', '<U10'), ('ejecta', 'f8'), ('wind', 'f8')])[:284]
        #  [:284] is to remove the unstable isotopes
        # y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        y = pd.DataFrame(y['ejecta']+y['wind'], index=y['isotope'], columns=[m_ini])
        # y = pd.DataFrame(y['ejecta'], index=y['isotope'], columns=[m_ini])
    y.index = y.index.str.capitalize()
    y['element'] = y.index.str.extract(r'([A-Za-z]+)').to_numpy().ravel()  # The element symbol
    y['A'] = y.index.str.extract(r'(\d+)').to_numpy().ravel()  # The mass number
    y[m_ini] = y[m_ini]/y['A'].astype(float)  # The yield in number
    y = y.groupby('element')[m_ini].sum().reset_index()
    y['Z'] = y['element'].apply(pyatomdb.atomic.elsymb_to_Z)
    y.sort_values(by='Z', inplace=True)
    y.set_index('element', inplace=True)
    y.drop('Z', axis=1, inplace=True)
    y.reset_index(inplace=True)
    return y


def merge_two_dfs(left, right):
    """
    Define a function to merge two DataFrames
    """
    return pd.merge(left, right, how='outer')

# From Tuguldur Sukhbold:

This being said, I can address your specific question - when considering different engines we have segregated the stars into two categories:

- $M_{\mathrm{ZAMS}}\leq 12 M_{\mathrm{\odot}}$: we considered them as "Crab like"
- $M_{\mathrm{ZAMS}}> 12 M_{\mathrm{\odot}}$: we considered them as "87A like"

for stars with $\leq 12 M_{\mathrm{\odot}}$ we only applied the Z9.6 engine, and for heavier stars we applied 4 different engines (W18, N20, S19.8 etc...) to get the explosion outcomes. And when computing integrated yields we combine Z9.6 results for lighter stars with W18 or N20 engine results of heavier stars, e.g.:

- combination 1: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + W18 (for $> 12 M_{\mathrm{\odot}}$)
- combination 2: Z9.6 (for $\leq 12 M_{\mathrm{\odot}}$) + N20 (for $> 12 M_{\mathrm{\odot}}$)

In [5]:
S16mods = ["W18", "N20"]  # The yields in Z9.6 have been copied and pasted to the W18 and N20 folder.
for S16mod in S16mods:
    files = os.listdir('./Original/S16/%s/' % S16mod)
    files = [item for item in files if item.endswith('.yield_table')]
    tables = [ ]
    for file in files:
        tables.append(get_S16_single_yields('./Original/S16/%s/%s' % (S16mod, file)))
    files = os.listdir('./Original/S16/Z9.6/')
    files = [item for item in files if item.endswith('.yield_table')]
    for file in files:
        tables.append(get_S16_single_yields('./Original/S16/%s/%s' % ('Z9.6', file)))
    merged_df = reduce(merge_two_dfs, tables)
    merged_df.set_index('element', inplace=True) 
    merged_df.reset_index(inplace=True)
    cols = merged_df.columns.drop('element')
    cols1 = np.array([float(item) for item in cols])
    cols = cols[cols1.argsort()]
    cols = cols.insert(0, 'element')
    merged_df = merged_df[cols]
    merged_df.to_csv('./SNcc/S16-%s.csv' % S16mod, index=False, float_format='%.2e')
    # The float format of original yield table is '%.2e'.